# Storing Machine Learning Run Metadata Using sqlite

Author: Travis Jefferies<br>
Last Updated: 05/07/2019<br>

This notebook walks through the creation of a flexible relational model that can be used to store metadata related to a given machine learning train/deployment run.  The relational model is then implemented in sqlite using a parallel ETL approach where data is stored in memory for on-demand processing needs downstream during training/deployment runs and archived on disk for reproducibility/audit trail purposes. Storing as much detail as possible about a given machine learning model run is necessary for model relevancy, metric tracking, and model assessment. Other pros/cons of this implementation technique are also explained.

## Import Libraries

In [1]:
import sqlalchemy
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
from functools import partial
np.random.seed(0)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.externals import joblib
from datetime import datetime
import time
import os
import zipfile

In [2]:
def create_connection():
    """ create a database connection to a SQLite database """
    try:
        conn = sqlite3.connect(':memory:')
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()

## Load data

In [3]:
features, target = make_classification(n_samples=1000, n_features=15, n_informative=6, n_classes=20)
df = pd.concat([pd.DataFrame(features), pd.Series(target,name='target')], axis=1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,target
0,0.481909,1.736034,-0.744777,-0.444399,-0.243341,1.468105,0.335756,2.043661,-0.802371,2.562580,2.591761,-0.122900,-1.542755,-0.152473,1.436250,3
1,0.328469,-0.182103,0.981706,1.426025,0.024267,-1.369246,-0.367076,-0.287884,0.109971,0.681591,0.858579,1.175316,-1.926863,0.778596,0.169588,19
2,-0.667993,-0.544896,-1.870499,-1.087367,3.878025,0.247470,0.731654,-0.467665,0.181174,-2.581402,1.850208,-0.707648,1.252330,0.213449,3.042264,0
3,0.320857,-1.378812,-0.040409,2.540748,-3.109547,-0.003694,-0.644819,1.227046,1.773983,0.074522,-1.186576,0.357030,1.000647,1.527006,-2.049990,15
4,1.944246,0.101500,-1.067252,-0.680489,1.054587,-1.165969,1.134462,4.217619,0.435200,4.040406,0.538467,1.125189,2.910772,-0.817434,-2.753014,0


# Split into train, test set

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('target',axis=1), df['target'], test_size=0.2)

## Store current time as `str`

We'll use this later in a variety of ways.

In [5]:
now = datetime.now()
now = now.strftime("%Y%m%d%H%M")
now

'201905101901'

## Train model and tune hyper parameters using `GridSearchCV`

In [6]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier(random_state=0)

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(X_train, y_train)
train_untuned_accuracy = clf.score(X_test, y_test)
print('rfc untuned accuracy: {}'.format(train_untuned_accuracy))

param_grid = { 
    'n_estimators': [20, 40],
    'max_features': ['auto', 'log2'],
    'max_depth': [10,20]
}


t = time.asctime( time.localtime(time.time()) )
CV_rfc = GridSearchCV(estimator=clf, param_grid=param_grid, cv=10)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)
CV_rfc.refit
e = time.asctime( time.localtime(time.time()) )
train_tuned_accuracy = CV_rfc.score(X_test, y_test)
print('tuned accuracy: {}'.format(train_tuned_accuracy))

rfc untuned accuracy: 0.295
{'max_depth': 20, 'max_features': 'auto', 'n_estimators': 40}
tuned accuracy: 0.315


## `SQLite` class

The `SQLite` class is used to create the sqlite database and execute queries.<br>
Under the hood, it uses `pandas` and `sqlite3` python libraries.

In [7]:
class SQLite:

    def __init__(self, db=None):
        """
        
        """
        if db:
            assert isinstance(db, str)
            assert db.split('.')[1] == 'db'
        self.create_connection(db)
    
    
    def create_connection(self, db=None):
        """ create a database connection to a SQLite database """
        try:
            if db:
                self.conn = sqlite3.connect(db)
                print(sqlite3.version)
            else:
                self.conn = sqlite3.connect(':memory:')
                print(sqlite3.version)
        except Error as e:
            print(e)
    
    def close_conn(self):
        self.conn.close()
            
def query_sqlite_db(conn, query):
    """
    
    """
    try:
        cur = conn.cursor()    
        cur.execute(query)
    except Error as e:
        print(e)
    finally:
        cur.close()

# Model metadata

In [8]:
create_sql = """CREATE TABLE Model(id INTEGER PRIMARY KEY, name TEXT, type TEXT, start_dt FLOAT, end_dt FLOAT)"""
insert_sql = """INSERT INTO Model VALUES({},'{}','{}','{}','{}')""".format(now, type(CV_rfc.estimator).__name__ ,str(type(CV_rfc.estimator))[8:-2],t,e)

s = SQLite()
query_sqlite_db(s.conn, create_sql)
query_sqlite_db(s.conn, insert_sql)
df = pd.read_sql_query('select * from Model',s.conn)

2.6.0


In [9]:
df.head()

,id,name,type,start_dt,end_dt
0,201905101901,RandomForestClassifier,sklearn.ensemble.forest.RandomForestClassifier,Fri May 10 19:01:50 2019,Fri May 10 19:01:57 2019


# Extending the concept to .pkl files

Now let's extend the concept from above to include .pkl files generated during the machine learning lifecycle. We'll be using the `cv_results_` attribute of the `GridSearchCV` object to illustrate.

In [10]:
CV_rfc.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [11]:
import pickle

# reference: https://stackoverflow.com/a/2340858

pdata0 = pickle.dumps(CV_rfc.best_params_, pickle.HIGHEST_PROTOCOL)
pdata1 = pickle.dumps(CV_rfc.cv_results_['params'], pickle.HIGHEST_PROTOCOL)
pdata2 = pickle.dumps(CV_rfc.cv_results_['mean_test_score'], pickle.HIGHEST_PROTOCOL)
pdata3 = pickle.dumps(CV_rfc.cv_results_['mean_train_score'], pickle.HIGHEST_PROTOCOL)
pdata4 = pickle.dumps(CV_rfc.cv_results_['mean_fit_time'], pickle.HIGHEST_PROTOCOL)
pdata5 = pickle.dumps(CV_rfc.cv_results_['mean_score_time'], pickle.HIGHEST_PROTOCOL)
pdata6 = pickle.dumps(CV_rfc.best_estimator_, pickle.HIGHEST_PROTOCOL)

C:\Users\jeffe\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [12]:
create_ModelGeneral_sql = """CREATE TABLE ModelTrainCV(id INTEGER PRIMARY KEY, name TEXT, type TEXT, start_dt TEXT, end_dt TEXT, optimal_model_params BLOB, all_models_params BLOB, all_models_test_scores BLOB, all_models_train_scores BLOB, all_models_fit_time_secs BLOB, all_models_score_time_secs BLOB, optimal_model BLOB)"""
curr = s.conn.cursor()
curr.execute(create_ModelGeneral_sql)
curr.execute("INSERT INTO ModelTrainCV VALUES (?,?,?,?,?,?,?,?,?,?,?,?)",(now, type(CV_rfc.estimator).__name__, str(type(CV_rfc.estimator))[8:-2], t, e, sqlite3.Binary(pdata0), sqlite3.Binary(pdata1), sqlite3.Binary(pdata2), sqlite3.Binary(pdata3), sqlite3.Binary(pdata4), sqlite3.Binary(pdata5), sqlite3.Binary(pdata6)))
df = pd.read_sql_query('select * from ModelTrainCV',s.conn)

In [13]:
df.head()

,id,name,type,start_dt,end_dt,optimal_model_params,all_models_params,all_models_test_scores,all_models_train_scores,all_models_fit_time_secs,all_models_score_time_secs,optimal_model
0,201905101901,RandomForestClassifier,sklearn.ensemble.forest.RandomForestClassifier,Fri May 10 19:01:50 2019,Fri May 10 19:01:57 2019,b'\x80\x04\x95:\x00\x00\x00\x00\x00\x00\x00}\x...,b'\x80\x04\x95\xb3\x00\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xca\x00\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xca\x00\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xca\x00\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xca\x00\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1f;\x02\x00\x00\x00\x00\x00\x8...


In [14]:
model = pickle.loads(df['optimal_model'][0])

In [15]:
assert np.allclose(model.predict(X_test), CV_rfc.predict(X_test))
print('success!')

success!


## Write in memory database to disk

In [16]:
s.conn.commit()

# write database to disk

c2 = sqlite3.connect('mydb.db')
with c2:
    for line in s.conn.iterdump():
        if line not in ('BEGIN;', 'COMMIT;'): # let python handle the transactions
            c2.execute(line)
c2.commit()

In [17]:
s.close_conn()
c2.close()

# Zip and remove

In [18]:
def zip_database(zip_file_path, sqlite_db_file_path):
    """Function to dump sqlite3 database into zipped file
    see: http://bit.ly/IAUmKc
    python 2.6+ only
    connection: sqlite3 database connection
    dumpfile: the name of the zipfile to store the sql_file info 
              [remember to add a .zip extension]
    sql_file: The name of the sql file to dump the database info into
              [remember to add a .sql extension]
    """
    
    zf = zipfile.ZipFile(zip_file_path, mode='w', compression = zipfile.ZIP_DEFLATED)

    # Create a zip file and write add the dump into it as a new file
    zf.write(os.getcwd()+'\\'+sqlite_db_file_path)
    zf.close()

In [19]:
zip_database('test2.zip', 'mydb.db')

In [20]:
if os.path.isfile(os.getcwd()+'\\'+'mydb.db'):
    os.remove(os.getcwd()+'\\'+'mydb.db')
else:    ## Show an error ##
    print("Error: %s file not found" % os.getcwd()+'\\'+'mydb.db')